In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import re
from io import StringIO
import PyPDF2
import pandas as pd
from datetime import datetime 
import datetime as dt
from timeit import default_timer as timer
pd.set_option('display.max_rows', 500)

### Nov

In [2]:
folder = "../Nov2020"
resultFolder = "../Nov2020_Result"
file = "18.11 3207971832_2020-November_eStatement.pdf"
path = folder + '/' + file
excelFile = resultFolder + '/' + re.sub('\.', '_', file.split('-')[0]) + '.xlsx'
excelFile

'../Nov2020_Result/18_11 3207971832_2020.xlsx'

In [3]:
date = re.sub('\.', '/', file.split(' ')[0])
previousDate = (datetime.strptime(date, "%d/%m") - dt.timedelta(days=1)).strftime('%d/%m')
#for nov
date = datetime.strptime(date, "%d/%m").strftime('%d/%m')
date, previousDate

('18/11', '17/11')

In [4]:
def convertPdfToTxt(path):
    startTime = timer()
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    #codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages,caching = caching, check_extractable = True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return text

text = convertPdfToTxt(path)

#extract no pages
noPage = len(re.findall("NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", text))

noPage

Total time: 108.4332s


1114

In [5]:
accountInfoDict = {}
accountCol = ["Nombor Akaun / Account Number", 'Jenis Akaun / Account Type',
             'Tarikh Penyata / Statement Date', 'RINGKASAN / SUMMARY']
for no, desc in enumerate(accountCol):
    if no != len(accountCol)-1:
        descIndex = text.index(desc)
        descLen = len(desc)
        nextDescIndex = text.index(accountCol[no + 1])
        result = text[descIndex + descLen: nextDescIndex]
        accountInfoDict[desc] = result
accountInfoDict

{'Nombor Akaun / Account Number': '3207971832',
 'Jenis Akaun / Account Type': 'RM Plus Current Account',
 'Tarikh Penyata / Statement Date': '18 Nov 2020'}

In [6]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

def extractAmount(desc, greedy = True):
#     greedyPattern = '([\d\.\,]+)'
    greedyPattern = '([\d\.\,]+\.\d{2})'
    notGreedyPattern = '([\d\.\,]+?\.\d{2}?)'
    try:
        if greedy:
            result = re.search(greedyPattern, desc).group(1)
            return result
        else:
            result = re.search(notGreedyPattern, desc).group(1)
            return result
    except Exception as e:
#         print(e)
        print("Wrong number pattern")

In [7]:
summaryDict = {}
summaryCol = ['Baki Penutup / Closing Balance',
              'Jumlah Debit / Total Debits', 'Bil. Debit / No. of Debits',
              'Jumlah Kredit / Total Credits', 'Bil. Kredit / No. of Credits']
#extract series of number 
numberSeries = extractText(text, summaryCol[4], 'Muka Surat 1')[0]

#extract closing balance amount
closingBalanceAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[0]] = closingBalanceAmount
numberSeries = re.sub(closingBalanceAmount, '', numberSeries)

#extract debit amount
debitAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[1]] = debitAmount
numberSeries = re.sub(debitAmount, '', numberSeries)

#credit no credit
noCredit = re.sub(extractAmount(numberSeries, greedy = False), '', numberSeries)
summaryDict[summaryCol[4]] = noCredit
numberSeries = re.sub(noCredit + '$', '', numberSeries)

tempNumberSeries = numberSeries
creditAmount1 = re.search('(\d{2}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit1 = re.sub(creditAmount1, '', tempNumberSeries)
creditAmount2 = re.search('(\d{1}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit2 = re.sub(creditAmount2, '', tempNumberSeries)

noDebitList = [noDebit1, noDebit2]
creditAmountList = [creditAmount1, creditAmount2]
noDebitList, creditAmountList

(['12014', '120141'], ['11,404,686.16', '1,404,686.16'])

In [8]:
def returnTableList(no):
    tempList = []
    textCopy = text
    for i in range(1, no + 1):
#     for i in range(1, 3):
        table, remain = extractText(textCopy, 'Page %s of %s'%(i, no), 'Penyata ini dicetak melalui komputer.')
        index = remain.index('Penyata ini dicetak melalui komputer.')
        termLen = len('Penyata ini dicetak melalui komputer.')
        textCopy = remain[index + termLen:]
        table = re.sub("TARIKHURUS NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", '', table)
        tempList.append(table)
    return tempList

tableList = returnTableList(noPage)

len(tableList)

1114

In [46]:
#extract first line of transaction

#trx date, cheque no., ref1, ref2, ref3, ref4, credit amount, debit amount, balance
resultList = []

#extract content of first table from text
table = tableList[0]

#extract balance amount
balanceAmount, tempText = extractText(table, "Balance From Last Statement", date)

#store record of balance from last statement
# resultList.append([previousDate, '','', '','', "Balance From Last Statement", '', '', balanceAmount])
#trx date, ref1, credit amount, debit amount, balance
resultList.append([previousDate, "Balance From Last Statement", '', '', balanceAmount])

In [47]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

In [48]:
#extract amountList and descList
#steps: 
#1. extract first and second number, store first number in descList
#2. extract desc between first and second number
#3. store desc in descList
#repeat 1 till 3 until closing balance's line

amountList = []
descList = []
closingLine = 0

# for no, table in enumerate([tableList[0]]):
for no, table in enumerate(tableList):
    if no == 0:
        tempText = tempText
    else:
        tempText = table
    # for i in range(3):
    while closingLine == 0:
        #remove date first
        if re.search(date, tempText):
            tempText = re.sub(date, '', tempText)
        #remove 9.60514075359E11: 412
        if no in [412, 943]:
            tempText = re.sub("9.60514075359E11", '', tempText)
        #extract amount first
        tempAmount1 = extractAmount(tempText)
        #store amount
        amountList.append(tempAmount1)
        #temporary subbed text to get second amount
        tempSubText = re.sub(tempAmount1, '', tempText)
        tempAmount2 = extractAmount(tempSubText)
        tempDesc = extractText(tempSubText, '', tempAmount2)
        
        #if Balance C/F, last line of transaction for that page
        if re.search(r"Balance C/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance C/F", "", str(tempDesc))
            amountList.append(tempAmount2)
            descList.append(tempDesc)
            descList.append("Balance C/F")
            break
            
        #if Balance B/F, first line of transaction for that page
        elif re.search(r"Balance B/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance B/F", "", str(tempDesc))
            descList.append("Balance B/F")
            tempAmount1 = extractAmount(re.sub(date, '', tempText), greedy = False)
            amountList[-1] = tempAmount1
            tempAmount1Index = tempText.index(tempAmount1)
            tempAmount1Len = len(tempAmount1)
            tempText = tempText[tempAmount1Index+tempAmount1Len:]
            
        #if last line for whole file
        elif re.search("Closing Balance In This Statement", str(tempDesc)):
            closingAmount = tempAmount2
            closingAmountIndex = tempSubText.index(closingAmount)
            lastDesc = tempSubText[:closingAmountIndex]
            #sub closing amount with '' in lastDesc
            lastDesc = re.sub("Closing Balance In This Statement", '', lastDesc)
            descList.append(lastDesc)

            #store last line - amount and desc
            amountList.append(closingAmount)
            descList.append("Closing Balance In This Statement")
#             closingLine = 1
            break
        #extracted all ady
        elif not tempAmount2:
#             closingLine = 1
            break
        else:
            tempText = re.sub('^' + re.escape(tempDesc), '', tempSubText)
            descList.append(tempDesc)

In [21]:
tableList[412]

'18/11Balance B/F24,744,337.78KEK BOON KIAT36.6724,744,301.11DR-ECP 0017733207971832PM18112001 MERCHANT REDEEMMERCHANT REDEEM 15112020 TOO BENG HAN1,100.0024,743,201.11MISC DR 0017733207971832PM18112099 EU REFUNDEU REFUND 11112020 JEFFREY PAU JION1,400.0024,741,801.11DR-ECP 0017733207971832PM18112097 COP REFUNDCOP REFUND 13112020 CHUA CHIE LIN1,525.1024,740,276.01MISC DR 0017733207971832PM18112002 MERCHANT REDEEMMERCHANT REDEEM 14112020SOON HONG BEST T0.1024,740,275.91HANDLING CHRG 001774FEE 3207971832PM18112098COP REFUND COP REFUND 15112020WOO PENG LUKE0.1024,740,275.81HANDLING CHRG 001775FEE 3207971832PM18112098COP REFUND COP REFUND 13112020YOO HUI SEE25.9724,740,249.84DR-ECP 0017753207971832PM18112001 MERCHANT REDEEMMERCHANT REDEEM 15112020GAM TONG (HONG K74.3824,740,175.46MISC DR 0017753207971832PM18112002 MERCHANT REDEEMMERCHANT REDEEM 14112020B AND G CENTRE99.0024,740,076.46MISC DR 0017753207971832PM18112099 EU REFUNDEU REFUND 11112020 9.60514075359E1110,000.0024,730,076.46DR-ECP

In [26]:
tableList[943]

'18/11Balance B/F21,086,794.310.1021,086,794.21HANDLING CHRG 003632FEE 3207971832PM18112099EU REFUND EU REFUND 111120209.60514075359E110.1021,086,794.11HANDLING CHRG 003633FEE 3207971832PM18112001MERCHANT REDEEM MERCHANT REDEEM13112020 KANG CHOI YANG0.1021,086,794.01HANDLING CHRG 003633FEE 3207971832PM18112097COP REFUND COP REFUND 13112020CHAN WAI HOONG0.1021,086,793.91HANDLING CHRG 003633FEE 3207971832PM18112099EU REFUND EU REFUND 11112020TAN CHEAH EN0.1021,086,793.81HANDLING CHRG 003634FEE 3207971832PM18112001MERCHANT REDEEM MERCHANT REDEEM13112020 LEE & LIEW POULT0.1021,086,793.71HANDLING CHRG 003634FEE 3207971832PM18112097COP REFUND COP REFUND 13112020HONG CHEE KEAT0.1021,086,793.61HANDLING CHRG 003634FEE 3207971832PM18112099EU REFUND EU REFUND 11112020WONG SIEW KEE0.1021,086,793.51HANDLING CHRG 003635FEE 3207971832PM18112001MERCHANT REDEEM MERCHANT REDEEM13112020 AH LOK TRADING0.1021,086,793.41HANDLING CHRG 003635FEE 3207971832PM18112097COP REFUND COP REFUND 13112020TAN BOON LIANG

In [49]:
def splitAmount(desc):
    try:
        #remove first . before num if exists
        desc = re.sub('^\.', '', desc)
        #remove date pattern
        desc = re.sub('^\d{1,2}\.\d{1,2}\.\d{4}', '', desc)
        amountPattern = '(.*?\.\d{2}?)'
        result = re.search(amountPattern, desc).group(1)
        desc2 = re.sub('^' + result, '', desc)
        return result, desc2
    except Exception as e:
        print(e)
        print("Wrong amount pattern")

splitAmountList = [splitAmount(i) for i in amountList]

In [50]:
def checkCreditDebit(amountTuple, previousBalance):
    #exception:
    amount1 = float(re.sub(',', '', amountTuple[0]))
#     currentBalance = re.sub('^\.', '', amountTuple[1])
    currentBalance = amountTuple[1]
#     print(currentBalance)
    if currentBalance !='':
        currentBalance = float(re.sub(',', '', currentBalance))
        previousBalance = float(re.sub(',', '', previousBalance))
        if currentBalance - previousBalance == amount1:
            amountTuple = ('', round(amount1, 2), currentBalance)
        elif previousBalance - currentBalance == amount1:
            amountTuple = (round(amount1, 2), '', currentBalance)
        #if amount 1 is wrong
        elif currentBalance > previousBalance:
            credit  = currentBalance - previousBalance
            amountTuple = ('', round(credit,2), currentBalance)
        elif previousBalance > currentBalance:
            debit  = previousBalance - currentBalance
            amountTuple = (round(debit,2), '', currentBalance)            
    else:
        amountTuple = ('', '', amount1)
    return amountTuple, str(amountTuple[2])

#categorize one amount as credit/debit
splitAmountList2 = []
for no, i in enumerate(splitAmountList):
    if no == 0:
        resultTuple, previousBalanceAmount = checkCreditDebit(i, balanceAmount)
        splitAmountList2.append(resultTuple)
    else:
        resultTuple, previousBalanceAmount = checkCreditDebit(i, previousBalanceAmount)
        splitAmountList2.append(resultTuple)


In [62]:
splitAmountList[723]

('104365.02', ',912.0039,737,571.73')

In [63]:
descList[723]

'MISC DR 0002013207971832PM11112098 COP REFUNDCOP REFUND 6112020 LAU SIEW PING'

In [64]:
tableList[52]

'11/11Balance B/F39,741,215.240.1039,741,215.14HANDLING CHRG 000200FEE 3207971832PM11112001COP REFUND COP REFUND 8112020MOHD ASHRAF BIN0.1039,741,215.04HANDLING CHRG 000201FEE 3207971832PM11112001COP REFUND COP REFUND 8112020MUHAMMAD HASAN B15.1139,741,199.93MISC DR 0002013207971832PM11112003 MERCHANT REDEEMMERCHANT REDEEM 6112020 MAH POOI SAN115.2039,741,084.73DR-ECP 0002013207971832PM11112002 MERCHANT REDEEMMERCHANT REDEEM 6112020 LING TAI NIK601.0039,740,483.73MISC DR 0002013207971832PM11112099 EU REFUNDEU REFUND 4112020 104365.02,912.0039,737,571.73MISC DR 0002013207971832PM11112098 COP REFUNDCOP REFUND 6112020 LAU SIEW PING3,732.0039,733,839.73MISC DR 0002013207971832PM11112004 MERCHANT REDEEMMERCHANT REDEEM 7112020 FONG YIN LEN0.1039,733,839.63HANDLING CHRG 000202FEE 3207971832PM11112001COP REFUND COP REFUND 8112020MOHD ASHRAF BIN0.1039,733,839.53HANDLING CHRG 000203FEE 3207971832PM11112001COP REFUND COP REFUND 8112020TEY CHIN TIANN42.0339,733,797.50DR-ECP 0002033207971832PM11112

In [51]:
colList = ['Trn. Date','Reference 1',
           'Debit Amount', 'Credit Amount', 'Balance']
for no, finalStrList in enumerate(descList):
    tempList =  [date] + [finalStrList]
    tempList +=  [i for i in splitAmountList2[no]]
    resultList.append(tempList)

df = pd.DataFrame(resultList, columns = colList)
df

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
0,17/11,Balance From Last Statement,,,"24,989,150.30"
1,18/11,SI CR FROM 3999XXXXXX,,1.11459e+07,3.6135e+07
2,18/11,RTN ITEM 091349IBG RTN ITEM 20201118 LTCS17201...,,65,3.61351e+07
3,18/11,RTN ITEM 091354IBG RTN ITEM 20201118 LTCS16201...,,2670,3.61378e+07
4,18/11,RTN ITEM 091355IBG RTN ITEM 20201118 LTCS16201...,,1069,3.61388e+07
...,...,...,...,...,...
14493,18/11,INSTANT TRSF DR 176942LIGHTREFT STUDIO PROGRAM...,25000,,2.03603e+07
14494,18/11,TSFR FUND DR-ATM/EFT 176946IBG TRANSFER - MBB ...,300,,2.036e+07
14495,18/11,INSTANT TRSF DR 176973 SU CHIA TEPROGRAM FEE,7000,,2.0353e+07
14496,18/11,TSFR FUND DR-ATM/EFT 1769824700XXXXXX SU CHIA-...,8000,,2.0345e+07


In [23]:
max([i for i in df['Debit Amount'] if type(i)!=str])

90033415.49

In [24]:
df.query("`Debit Amount`==90033415.49")

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
12367,18/11,HANDLING CHRG 003633FEE 3207971832PM18112001ME...,9.00334e+07,,2.10868e+07


In [25]:
df.query("`Debit Amount`==90033415.49")['Reference 1'].values

array(['HANDLING CHRG 003633FEE 3207971832PM18112001MERCHANT REDEEM MERCHANT REDEEM13112020 KANG CHOI YANG'],
      dtype=object)

In [52]:
df.loc[df.shape[0]-1]['Balance']

20344996.57

In [53]:
sum([i for i in df['Debit Amount'] if i!=''])

16048839.889997806

In [54]:
trueNoDebit = df.query("`Debit Amount`!=''").shape[0]
trueNoDebit

12014

In [55]:
sum([i for i in df['Credit Amount'] if i!=''])

11404686.16

In [56]:
#confirm no debit
noDebitIndex = noDebitList.index(str(trueNoDebit))
creditAmount = creditAmountList[noDebitIndex]
summaryDict[summaryCol[2]] = trueNoDebit
summaryDict[summaryCol[3]] = creditAmount

summaryDict

{'Baki Penutup / Closing Balance': '20,344,996.57',
 'Jumlah Debit / Total Debits': '16,048,839.89',
 'Bil. Kredit / No. of Credits': '256',
 'Bil. Debit / No. of Debits': 12014,
 'Jumlah Kredit / Total Credits': '11,404,686.16'}

In [57]:
import xlsxwriter 
  

workbook = xlsxwriter.Workbook(excelFile)
worksheet = workbook.add_worksheet() 

# Rows and columns are zero indexed. 
row = 0

for col in accountInfoDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, accountInfoDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
for col in summaryDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, summaryDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
workbook.close()

from openpyxl import load_workbook
#write pandas 
writer = pd.ExcelWriter(excelFile, engine='openpyxl')
writer.book = load_workbook(excelFile)
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(excelFile)
df.to_excel(writer, sheet_name='Sheet1', startrow = len(reader) + 2, header = True, index = False)
writer.save()

### final page

In [39]:
#extract amountList and descList
#steps: 
#1. extract first and second number, store first number in descList
#2. extract desc between first and second number
#3. store desc in descList
#repeat 1 till 3 until closing balance's line

amountList = []
descList = []
closingLine = 0

for no, table in enumerate([tableList[-1]]):
# for no, table in enumerate(tableList[:-1]):
    if no == 0:
        tempText = tempText
    else:
        tempText = table
    # for i in range(3):
    while closingLine == 0:
        #remove date first
        if re.search(date, tempText):
            tempText = re.sub(date, '', tempText)
        #remove RMB @1.55
        tempText = re.sub("RMB.*@ \d\.\d+", '', tempText)
        if no in [469, 839]:    
        #remove 9.10726135944E11: table 469, 839
            tempText = re.sub("9.10726135944E11", '', tempText)
        #extract amount first
        tempAmount1 = extractAmount(tempText)
        #store amount
        amountList.append(tempAmount1)
        #temporary subbed text to get second amount
        tempSubText = re.sub(tempAmount1, '', tempText)
        tempAmount2 = extractAmount(tempSubText)
        tempDesc = extractText(tempSubText, '', tempAmount2)
        
        #if Balance C/F, last line of transaction for that page
        if re.search(r"Balance C/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance C/F", "", str(tempDesc))
            amountList.append(tempAmount2)
            descList.append(tempDesc)
            descList.append("Balance C/F")
            break
            
        #if Balance B/F, first line of transaction for that page
        elif re.search(r"Balance B/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance B/F", "", str(tempDesc))
            descList.append("Balance B/F")
            tempAmount1 = extractAmount(re.sub(date, '', tempText), greedy = False)
            amountList[-1] = tempAmount1
            tempAmount1Index = tempText.index(tempAmount1)
            tempAmount1Len = len(tempAmount1)
            tempText = tempText[tempAmount1Index+tempAmount1Len:]
            
        #if last line for whole file
        elif re.search("Closing Balance In This Statement", str(tempDesc)):
            closingAmount = tempAmount2
            closingAmountIndex = tempSubText.index(closingAmount)
            lastDesc = tempSubText[:closingAmountIndex]
            #sub closing amount with '' in lastDesc
            lastDesc = re.sub("Closing Balance In This Statement", '', lastDesc)
            descList.append(lastDesc)

            #store last line - amount and desc
            amountList.append(closingAmount)
            descList.append("Closing Balance In This Statement")
#             closingLine = 1
            break
        #extracted all ady
        elif not tempAmount2:
#             closingLine = 1
            break
        else:
            tempText = re.sub('^' + re.escape(tempDesc), '', tempSubText)
            descList.append(tempDesc)

In [43]:
tableList[-2]

"18/11Balance B/F20,430,803.87JOMPAY - 7BI45N5B - PENGURUSAN AIR PAHAN30.8020,430,773.07GIRO PYMT-ATM/EFT 090235JOMPAY - 7BI2WRC6 - PENGURUSAN AIR PAHAN10.0020,430,763.07GIRO PYMT-ATM/EFT 090236JOMPAY - 7BIGE4V9 - PENGURUSAN AIR PAHAN100.0020,430,663.07GIRO PYMT-ATM/EFT 090238JOMPAY - 7BI4TJKA - PENGURUSAN AIR PAHAN6.7020,430,656.37GIRO PYMT-ATM/EFT 090247JOMPAY - 7BIKS1P2 - PENGURUSAN AIR PAHAN200.0020,430,456.37GIRO PYMT-ATM/EFT 090249JOMPAY - 7BITHDWM - PENGURUSAN AIR PAHAN17.0020,430,439.37GIRO PYMT-ATM/EFT 090257JOMPAY - 7BIM9FSM - PENGURUSAN AIR PAHAN21.6520,430,417.72GIRO PYMT-ATM/EFT 090260JOMPAY - 7BISI8E6 - PENGURUSAN AIR PAHAN20.0020,430,397.72GIRO PYMT-ATM/EFT 090280JOMPAY - 7BIF0EJ6 - PENGURUSAN AIR PAHAN6,400.0020,423,997.72INSTANT TRSF DR 090292 ALAN ALBERT AUCOP REFUND 0811202016.1520,423,981.57GIRO PYMT-ATM/EFT 090306JOMPAY - 7BIS1KV8 - PENGURUSAN AIR PAHAN4,141.0020,419,840.57INSTANT TRSF DR 090332KWEEK WOEI LUAN COP REFUND 0911202010,000.0020,409,840.57INSTANT TRSF D

In [42]:
tableList[-1]

"18/11Balance B/F20,385,296.5725,000.0020,360,296.57INSTANT TRSF DR 176942LIGHTREFT STUDIO PROGRAM FEE300.0020,359,996.57TSFR FUND DR-ATM/EFT 176946IBG TRANSFER - MBB RENTAL NOV 2020SINOPAY (MALAYSIA) S RFIV/2011-02507,000.0020,352,996.57INSTANT TRSF DR 176973 SU CHIA TEPROGRAM FEE8,000.0020,344,996.57TSFR FUND DR-ATM/EFT 1769824700XXXXXX SU CHIA-TE PROGRAM FEEClosing Balance In This Statement20,344,996.57Baki Harian Dan Penutup Meliputi Semua Cek Yang Telah Didepositkan, Dijelaskan Dan Yang Belum Dijelaskan.Terima Kasih Kerana Berurus Niaga Dengan Public Bank.  Kecemerlangan Adalah Iltizam Kami.Kemusykilan anda mengenai perbankan DIJAWAB! Untuk maklumat lanjut, sila layari www.bankinginfo.com.my.Daily And Closing Balances Include All Cheques Deposited, Cleared And Uncleared.Thank You For Banking With Public Bank.  Excellence Is Our Commitment.Your banking questions ANSWERED! For more info, log on to www.bankinginfo.com.my.Anda boleh melihat Notis Privasi Public Bank di laman web www.p

In [40]:
amountList

['400.0020,385,296.57', '20,385,296.57']

In [41]:
descList

['TSFR FUND DR-ATM/EFT 176941IBG TRANSFER - MBB RENTAL NOV 2020SINOPAY (MALAYSIA) S RFIV/2011-0249',
 'Balance C/F']